In [ ]:
def ideal_feat_num_rfe(data,num_f,cat_f,target_f,selec_model,selec_scaler,selec_sampler,n_folds):         

    binary_f=[]
    
    for feat in cat_f:
        
        unique_values=data[feat].unique()
        
        if len(unique_values)==2 and set(unique_values)<={0, 1}:
            
            cat_f=np.delete(cat_f,np.where(cat_f==feat))
            binary_f.append(feat)

    binary_f=np.array(binary_f)
    all_f=np.concatenate((cat_f,num_f,binary_f))

    nof_list=np.arange(1,len(all_f)+1)            
    high_score=0
    nof=0 
    
    for n in range(len(nof_list)):
        
        val_score_list=[]
        rank_list=[]
        skf=StratifiedKFold(n_splits=n_folds)
        
        for train_index,val_index in skf.split(data[all_f],data[target_f]):
            
            model=selec_model
            scaler=selec_scaler
            sampler=selec_sampler
            rfe=RFE(estimator=model,n_features_to_select=nof_list[n])
            
            X_train,X_val=data[all_f].iloc[train_index],data[all_f].iloc[val_index]
            y_train,y_val=data[target_f].iloc[train_index],data[target_f].iloc[val_index]

            
            
            if len(num_f)!=0:
                scaler.fit(X_train[num_f])
                X_train[num_f]=scaler.transform(X_train[num_f])
                X_val[num_f]=scaler.transform(X_val[num_f])


            if len(cat_f)!=0:
                
                encoding_map={}
                
                for col in cat_f:
                    freq_encoding=data.iloc[train_index].groupby(col)[target_f].mean()
                    encoding_map[col]=freq_encoding
                
                for col,encoding in encoding_map.items():
                    X_train[col]=X_train[col].map(encoding)
                
                for col,encoding in encoding_map.items():
                    X_val[col]=X_val[col].map(encoding)

            
            X_resampled,y_resampled=sampler.fit_resample(X_train,y_train)
            
            X_train_rfe=rfe.fit_transform(X_resampled,y_resampled)
            X_val_rfe=rfe.transform(X_val)
            model.fit(X_train_rfe,y_resampled)

            y_pred=model.predict(X_val_rfe)
            val_score_list.append(f1_score(y_val,y_pred))
            rank_list.append(rfe.ranking_)

        if(np.mean(val_score_list)>=high_score):
            high_score=np.mean(val_score_list)
            nof=nof_list[n]
            ranks=np.mean(rank_list,axis=0)
    
    ranks_series=pd.Series(ranks,index=data[all_f].columns)
    
    print("Optimum number of features: %d" %nof)
    print("Score with %d features: %f" % (nof, high_score))
    print("Feature ranking:")
    print(ranks_series)


def TestIndependence(X,y,var,alpha=0.05):

    dfObserved=pd.crosstab(y,X) 
    chi2,p,dof,expected=stats.chi2_contingency(dfObserved.values)
    dfExpected=pd.DataFrame(expected,columns=dfObserved.columns,index=dfObserved.index)
    Bool=1
    
    if p<alpha:
        result="{0} is IMPORTANT for Prediction".format(var)
    else:
        result="{0} is NOT an important predictor. (Discard {0} from model)".format(var)
        Bool=0
    print(result,"p=",p)
    return(Bool)

def model_fit(X,y,num_feats,model,scaler,sampler):

    X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3,random_state=0,stratify=y,shuffle=True)

    scaler.fit(X_train[num_feats])
    X_train[num_feats]=scaler.transform(X_train[num_feats])
    X_val[num_feats]=scaler.transform(X_val[num_feats])

    X_resampled,y_resampled=sampler.fit_resample(X_train,y_train)

    model.fit(X_resampled,y_resampled)

    y_pred=model.predict(X_val)
    val_score=f1_score(y_val,y_pred)

    return val_score
 